<a href="https://colab.research.google.com/github/JasmineElm/Notebooks/blob/master/grab_FREE_BFI_Titles_on_1001M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from tabulate import tabulate #to export dataframe as markdown table
import requests,urllib.request, json 

In [0]:
## grab our 1001 Movies List, download, and load into a dataframe 
url ='https://gist.githubusercontent.com/JasmineElm/ce8219c58bd416c0aec588a97e168221/raw/57717b4ae21f1721b2e1c22d2e8a74795e0e54d4/netflixTitles.csv'
r = requests.get(url)
filename = url.split('/')[-1]
 
with open(filename,'wb') as output_file:
    output_file.write(r.content)

ml=pd.read_csv(filename,index_col=[0])


In [136]:
# free films are split into collections.  each collection is housed on a separate page.
# build a list of these collection links.
collections=[]
url='https://player.bfi.org.uk/free/collections'
page = requests.get(url)
soup = BeautifulSoup(page.text, "html5lib")
base='https://player.bfi.org.uk'

for collection in soup.findAll('div',{'class':'collection-card'}):
  collections.append(base+collection.find('a').get('href'))

print(len(collections))

74


In [152]:
movieList=[]
movieInfo=[]
progress=len(collections)

for idx,collection in enumerate(collections):
  print(str(int(((len(collections)-idx)/len(collections)*100)))+'%\t',end='')
  sleep(randint(0,2)) #let's not spam the server
  page = requests.get(collection)
  soup = BeautifulSoup(page.text, "html5lib")
  for card in soup.findAll('div',{'class':'card--free'}):
    movieInfo=[]
    movieInfo.append(base+card.find('a').get('href')) #direct link  
    movieInfo.append(card.find('h3').find('span').text) #title
    if len(card.find('p',{'class':'card__info'}).select("span:nth-of-type(2)")[0].text)==4:
      movieInfo.append(card.find('p',{'class':'card__info'}).select("span:nth-of-type(2)")[0].text)
    else:
      movieInfo.append('0') #hacky way to ensure only years are parsed - needs fixing
    movieList.append(movieInfo)

100% 
98% 
97% 
95% 
94% 
93% 
91% 
90% 
89% 
87% 
86% 
85% 
83% 
82% 
81% 
79% 
78% 
77% 
75% 
74% 
72% 
71% 
70% 
68% 
67% 
66% 
64% 
63% 
62% 
60% 
59% 
58% 
56% 
55% 
54% 
52% 
51% 
50% 
48% 
47% 
45% 
44% 
43% 
41% 
40% 
39% 
37% 
36% 
35% 
33% 
32% 
31% 
29% 
28% 
27% 
25% 
24% 
22% 
21% 
20% 
18% 
17% 
16% 
14% 
13% 
12% 
10% 
9% 
8% 
6% 
5% 
4% 
2% 
1% 


In [153]:
#clean up our movie list
df=pd.DataFrame(data=movieList, columns=['Link','Title','Year']).dropna()
df['Year']=df['Year'].str[-4:]
df=df.dropna().astype({'Year': 'int64'})
#merge it with the 1001 movies list
on_nf = pd.merge(ml, df,  how='inner', left_on=['Title','Year'], right_on = ['Title','Year'])

#generate a nice markdown table that can be pasted into Reddit
if(on_nf.shape[0]>0):
  print(tabulate(on_nf.drop(['Metascore','Language','Country','IMDBRating', 'Plot','Awards','imdbID'],axis=1).sort_values('Title')
  , tablefmt="pipe", headers="keys", showindex=False))
else:
  print("no matches")

| Title              | Genre                             | Director       |   Year |   Runtime | Link                                                                     |
|:-------------------|:----------------------------------|:---------------|-------:|----------:|:-------------------------------------------------------------------------|
| A Trip to the Moon | Short, Adventure, Fantasy, Sci-Fi | Georges Méliès |   1902 |        13 | https://player.bfi.org.uk/free/film/watch-a-trip-to-the-moon-1902-online |
